In [1]:
#scrapping code for men shirt 

from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json
import logging
import concurrent.futures
import pandas as pd

# Set up logging
logging.basicConfig(level=logging.INFO)

def get_random_user_agent():
    ua = UserAgent()
    return ua.random

def scrape_product(shoe):
    try:
        name_element = WebDriverWait(shoe, 20).until(EC.presence_of_element_located((By.XPATH, ".//span[@class='a-size-base-plus a-color-base a-text-normal']")))
        name = name_element.text
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product name")
        name = "No name available"

    try:
        product_id = shoe.get_attribute("data-asin")
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product ID")
        product_id = ""

    try:
        price_element = shoe.find_element(By.XPATH, ".//span[@class='a-price-whole']")
        price = price_element.text if price_element else "0"
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product price")
        price = "No price available"

    try:
        product_url_element = shoe.find_element(By.XPATH, ".//a[@class='a-link-normal s-no-outline']")
        product_url = product_url_element.get_attribute("href") if product_url_element else ""
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product URL")
        product_url = ""

    try:
        brand_element = shoe.find_element(By.XPATH, ".//span[@class='a-size-base-plus a-color-base']")
        brand = brand_element.text.strip()
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product description")
        brand = "No description available"

    return {
        "product_name": name,
        "product_id": product_id,
        "price": price,
        "product_url": product_url,
        "brand_name": brand,
        "gender": "male"  # Hardcoded gender as female
    }

def main():
    # Use the Firefox driver
    options = webdriver.FirefoxOptions()
    driver = webdriver.Firefox(options=options)

    try:
        all_data = []  # List to store data from all pages

        for page_number in range(10, 60):  # Scraping 10 pages
            url = f"https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page={page_number}"
            logging.info(f"Scraping page {page_number}: {url}")

            # Setting random user agent for each request
            user_agent = get_random_user_agent()
            logging.info(f"Using user agent: {user_agent}")
            options.set_preference("general.useragent.override", user_agent)

            driver.get(url)
            driver.maximize_window()

            # Add a delay to allow the page to load
            time.sleep(2)

            # Adjusted XPath expression to capture all product elements
            shoes = driver.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')

            data = []

            # Concurrently scrape product information
            with concurrent.futures.ThreadPoolExecutor() as executor:
                results = executor.map(scrape_product, shoes)

            # Collect results
            for result in results:
                data.append(result)
                print(json.dumps(result, indent=4))

            all_data.extend(data)

            logging.info(f"Page {page_number} data collected")

        # Save all data to a single JSON file
        with open("menshirts1.json", "w") as json_file:
            json.dump(all_data, json_file, indent=4)
        logging.info("All data saved to amazon_shoes_women.json")

        # Convert all data to a DataFrame
        df = pd.DataFrame(all_data)
        df.to_csv("menshirts1.csv", index=False)
        logging.info("All data saved to menshoes1.csv")

    finally:
        driver.quit()

if __name__ == "__main__":
    main()


INFO:root:Scraping page 10: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=10
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; rv:102.0) Gecko/20100101 Firefox/102.0
INFO:root:Page 10 data collected
INFO:root:Scraping page 11: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=11
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.31
INFO:root:Page 11 data collected
INFO:root:Scraping page 12: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=12
INFO:root:Using user agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Safari/605.1.15


{
    "product_name": "Men's Collared Button Up Shirt for Casual Wear | Premium Stretchable Cotton Shirt Crafted with Regular Sleeve, Comfort Fit & High-Performance for Everyday Wear",
    "product_id": "B0BWRFPVSS",
    "price": "639",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1Nzc3ODM2NTg3MjEyMDUxOjE3MDg1MzE4MDY6c3BfYXRmX25leHQ6MjAxMjQ3MDkwMjY2OTg6OjA6Og&url=%2FCampus-Sutra-Multicolour-Checkered-High-Performance%2Fdp%2FB0BWRFPVSS%2Fref%3Dsr_1_481%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%26qid%3D1708531806%26sprefix%3Dshirts%252Caps%252C434%26sr%3D8-481-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "Campus Sutra",
    "gender": "male"
}
{
    "product_name": "Men Cotton Solid Full Sleeve Casual Shirt (Available in Plus Size)",
    "product_id": "B08JQS8VDK",
    "price": "599",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1Nzc3ODM2NTg3MjEyMDUxOjE3MDg1MzE4MDY6c3BfYXRmX25leHQ6MzAwMDQzNjI5NzY3N

INFO:root:Page 12 data collected
INFO:root:Scraping page 13: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=13
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.43


{
    "product_name": "Men's Collared Button Up Shirt for Casual Wear | Premium Stretchable Cotton Shirt Crafted with Regular Sleeve, Comfort Fit & High-Performance for Everyday Wear",
    "product_id": "B0BWRGSP81",
    "price": "640",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo0NjQ2NDMyNzgwNDE0MDUxOjE3MDg1MzE4MTk6c3BfYXRmX25leHQ6MjAxMjQ3MDkwMTk2OTg6OjA6Og&url=%2FCampus-Sutra-Checkered-Collared-High-Performance%2Fdp%2FB0BWRGSP81%2Fref%3Dsr_1_529%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%26qid%3D1708531819%26sprefix%3Dshirts%252Caps%252C434%26sr%3D8-529-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "Campus Sutra",
    "gender": "male"
}
{
    "product_name": "Men Cotton Solid Full Sleeve Casual Shirt (Available in Plus Size)",
    "product_id": "B0822PSL3C",
    "price": "549",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo0NjQ2NDMyNzgwNDE0MDUxOjE3MDg1MzE4MTk6c3BfYXRmX25leHQ6MzAwMDQzNjIzOTY0MzMy

INFO:root:Page 13 data collected
INFO:root:Scraping page 14: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=14
INFO:root:Using user agent: Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/117.0


...

{
    "product_name": "Mens 100% Cotton Full Sleeve Regular Fit Fashion Checkered Shirt",
    "product_id": "B0BVFVWT6S",
    "price": "719",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo3MTExNzc0NjI2MjMxMDUxOjE3MDg1MzIyMTY6c3BfYXRmX25leHQ6MzAwMTMzMzcyODIzNTMyOjowOjo&url=%2FGLORYBOYZ-Cotton-Regular-Fashion-Checkered%2Fdp%2FB0BVFVWT6S%2Fref%3Dsr_1_2689%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%26qid%3D1708532216%26sprefix%3Dshirts%252Caps%252C434%26sr%3D8-2689-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "GLORYBOYZ",
    "gender": "male"
}
{
    "product_name": "Racing Polo T Shirt Cotton Grey",
    "product_id": "B08P73NQP6",
    "price": "764",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo3MTExNzc0NjI2MjMxMDUxOjE3MDg1MzIyMTY6c3BfYXRmX25leHQ6MzAwMTMzMTA4NDY2NDMyOjowOjo&url=%2FTVS-Racing-Cotton-T-Shirt-Grey%2Fdp%2FB08P73NQP6%2Fref%3Dsr_1_2690%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen

INFO:root:Page 58 data collected
INFO:root:Scraping page 59: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=59
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.76


{
    "product_name": "Mens 100% Cotton Full Sleeve Regular Fit Fashion Checkered Shirt",
    "product_id": "B0BVFQNMVH",
    "price": "719",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1NzIwMjYyOTY5NDkyMDUxOjE3MDg1MzIyMjQ6c3BfYXRmX25leHQ6MzAwMTMzMzcyODIxNzMyOjowOjo&url=%2FGLORYBOYZ-Cotton-Regular-Fashion-Checkered%2Fdp%2FB0BVFQNMVH%2Fref%3Dsr_1_2737%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%26qid%3D1708532224%26sprefix%3Dshirts%252Caps%252C434%26sr%3D8-2737-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "GLORYBOYZ",
    "gender": "male"
}
{
    "product_name": "Racing Polo T Shirt Polyester Red",
    "product_id": "B08P327RWG",
    "price": "1,249",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1NzIwMjYyOTY5NDkyMDUxOjE3MDg1MzIyMjQ6c3BfYXRmX25leHQ6MzAwMTMzMTA4NDY0NTMyOjowOjo&url=%2FTVS-Polyester-T-Shirt-Red-T-shirts%2Fdp%2FB08P327RWG%2Fref%3Dsr_1_2738%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bf

INFO:root:Page 59 data collected
INFO:root:All data saved to amazon_shoes_women.json
INFO:root:All data saved to menshoes1.csv


{
    "product_name": "Men's Black Contrast Pocket Shirt",
    "product_id": "B0CN1CD2GL",
    "price": "999",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo2MjIyODQyMzYwMjE5MDUxOjE3MDg1MzIyMzM6c3BfYXRmX25leHQ6MzAwMTMwMzU1NTk4MzMyOjowOjo&url=%2FDON-VINO-Black-Contrast-Pocket%2Fdp%2FB0CN1CD2GL%2Fref%3Dsr_1_2785%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%26qid%3D1708532233%26sprefix%3Dshirts%252Caps%252C434%26sr%3D8-2785-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "DON VINO",
    "gender": "male"
}
{
    "product_name": "Men's Printed Sports Performance Tshirt for Running & Training",
    "product_id": "B0CMDGNBYM",
    "price": "759",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo2MjIyODQyMzYwMjE5MDUxOjE3MDg1MzIyMzM6c3BfYXRmX25leHQ6MzAwMTEzNTMxMTc3OTMyOjowOjo&url=%2FTRUEREVO-Printed-Performance-Training-191128CMPRBLU_S_Camo%2Fdp%2FB0CMDGNBYM%2Fref%3Dsr_1_2786%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshi

In [2]:
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json
import logging
import concurrent.futures
import pandas as pd

# Set up logging
logging.basicConfig(level=logging.INFO)

def get_random_user_agent():
    ua = UserAgent()
    return ua.random

def scrape_product(shoe):
    try:
        name_element = WebDriverWait(shoe, 20).until(EC.presence_of_element_located((By.XPATH, ".//span[@class='a-size-base-plus a-color-base a-text-normal']")))
        name = name_element.text
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product name")
        name = "No name available"

    try:
        product_id = shoe.get_attribute("data-asin")
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product ID")
        product_id = ""

    try:
        price_element = shoe.find_element(By.XPATH, ".//span[@class='a-price-whole']")
        price = price_element.text if price_element else "0"
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product price")
        price = "No price available"

    try:
        product_url_element = shoe.find_element(By.XPATH, ".//a[@class='a-link-normal s-no-outline']")
        product_url = product_url_element.get_attribute("href") if product_url_element else ""
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product URL")
        product_url = ""

    try:
        brand_element = shoe.find_element(By.XPATH, ".//span[@class='a-size-base-plus a-color-base']")
        brand = brand_element.text.strip()
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product description")
        brand = "No description available"

    return {
        "product_name": name,
        "product_id": product_id,
        "price": price,
        "product_url": product_url,
        "brand_name": brand,
        "gender": "male"  # Hardcoded gender as female
    }

def main():
    # Use the Firefox driver
    options = webdriver.FirefoxOptions()
    driver = webdriver.Firefox(options=options)

    try:
        all_data = []  # List to store data from all pages

        for page_number in range(70, 100): 
            url = f"https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page={page_number}"
            logging.info(f"Scraping page {page_number}: {url}")

            # Setting random user agent for each request
            user_agent = get_random_user_agent()
            logging.info(f"Using user agent: {user_agent}")
            options.set_preference("general.useragent.override", user_agent)

            driver.get(url)
            driver.maximize_window()

            # Add a delay to allow the page to load
            time.sleep(2)

            # Adjusted XPath expression to capture all product elements
            shoes = driver.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')

            data = []

            # Concurrently scrape product information
            with concurrent.futures.ThreadPoolExecutor() as executor:
                results = executor.map(scrape_product, shoes)

            # Collect results
            for result in results:
                data.append(result)
                print(json.dumps(result, indent=4))

            all_data.extend(data)

            logging.info(f"Page {page_number} data collected")

        # Save all data to a single JSON file
        with open("menshirts2.json", "w") as json_file:
            json.dump(all_data, json_file, indent=4)
        logging.info("All data saved to men shirts .json")

        # Convert all data to a DataFrame
        df = pd.DataFrame(all_data)
        df.to_csv("menshirts2.csv", index=False)
        logging.info("All data saved to mensirt1.csv")

    finally:
        driver.quit()

if __name__ == "__main__":
    main()


INFO:root:Scraping page 70: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=70
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0
INFO:root:Page 70 data collected
INFO:root:Scraping page 71: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=71
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.81
INFO:root:Page 71 data collected
INFO:root:Scraping page 72: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=72
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.69


{
    "product_name": "Men's Cotton Rich Solid Polo T-Shirt Regular Fit (Available in Plus Size)",
    "product_id": "B07MZKNLGF",
    "price": "445",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MToyNDYwMzYzOTIyNzYyMDUxOjE3MDg1MzI0MjQ6c3BfYXRmX25leHQ6MzAwMTE2ODI5NDE0NDMyOjowOjo&url=%2FAmazon-Brand-Symbol-Polo_SS19PLS2_Fog-Teal_Medium%2Fdp%2FB07MZKNLGF%2Fref%3Dsr_1_3361%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%26qid%3D1708532424%26sprefix%3Dshirts%252Caps%252C434%26sr%3D8-3361-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "Amazon Brand - Symbol",
    "gender": "male"
}
{
    "product_name": "Black Bull Men's Regular Fit Cotton Casual Shirt for Men Full Sleeves",
    "product_id": "B0CRHPZSVT",
    "price": "799",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MToyNDYwMzYzOTIyNzYyMDUxOjE3MDg1MzI0MjQ6c3BfYXRmX25leHQ6MzAwMTIxMTk0MzcyMTMyOjowOjo&url=%2FRegular-Cotton-Casual-Sleeves-Medium%2Fdp%2FB0CRHPZSVT%

INFO:root:Page 72 data collected
INFO:root:Scraping page 73: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=73
INFO:root:Using user agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36


{
    "product_name": "Black Bull Men's Regular Fit Cotton Casual Shirt for Men Full Sleeves",
    "product_id": "B0CRHQ6XWC",
    "price": "799",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1NTQ0NzU2NzEzMTQ3MDUxOjE3MDg1MzI0MzQ6c3BfYXRmX25leHQ6MzAwMTIxMTk0MzcwMjMyOjowOjo&url=%2FRegular-Cotton-Casual-Sleeves-Medium%2Fdp%2FB0CRHQ6XWC%2Fref%3Dsr_1_3409%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%26qid%3D1708532434%26sprefix%3Dshirts%252Caps%252C434%26sr%3D8-3409-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "Generic",
    "gender": "male"
}
{
    "product_name": "Men's Collared Button Up Shirt for Casual Wear | Premium Stretchable Cotton Shirt Crafted with Regular Sleeve, Comfort Fit & High-Performance for Everyday Wear",
    "product_id": "B0BWRFZBP3",
    "price": "645",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1NTQ0NzU2NzEzMTQ3MDUxOjE3MDg1MzI0MzQ6c3BfYXRmX25leHQ6MjAxMjQ3MDkwMjM3OTg6OjA6Og&url=

INFO:root:Page 73 data collected
INFO:root:Scraping page 74: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=74
INFO:root:Using user agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36


...

{
    "product_name": "Men's Formal Shirt",
    "product_id": "B0B5YD8QF2",
    "price": "1,039",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MToyODQwODk2MTY2ODk4MDUxOjE3MDg1MzI2NDA6c3BfYXRmX25leHQ6MzAwMTAzMDY2NTQ4NzMyOjowOjo&url=%2FVan-Heusen-Mens-Shirt-VHSFRSLBK44422_Pink%2Fdp%2FB0B5YD8QF2%2Fref%3Dsr_1_4609%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%26qid%3D1708532640%26sprefix%3Dshirts%252Caps%252C434%26sr%3D8-4609-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "Van Heusen",
    "gender": "male"
}
{
    "product_name": "Men Shirt",
    "product_id": "B0C9F1H63R",
    "price": "1,100",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MToyODQwODk2MTY2ODk4MDUxOjE3MDg1MzI2NDA6c3BfYXRmX25leHQ6MzAwMTA1MDkwMTM3MTMyOjowOjo&url=%2FLee-Mens-Slim-Shirt-LMSH003932_Rust%2Fdp%2FB0C9F1H63R%2Fref%3Dsr_1_4610%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%26qid%3D1708532640%26sprefix%3Dshirts%252Caps%252C434%26sr

INFO:root:Page 98 data collected
INFO:root:Scraping page 99: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=99
INFO:root:Using user agent: Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/118.0


{
    "product_name": "Men Shirt",
    "product_id": "B0C9DYNW5K",
    "price": "1,100",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo3NjYwNTk4ODg1MDg1MDUxOjE3MDg1MzI2NDg6c3BfYXRmX25leHQ6MzAwMTA1MDkwMTM3MzMyOjowOjo&url=%2FLee-Mens-Slim-Shirt-LMSH003937_Red%2Fdp%2FB0C9DYNW5K%2Fref%3Dsr_1_4657%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%26qid%3D1708532648%26sprefix%3Dshirts%252Caps%252C434%26sr%3D8-4657-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "Lee",
    "gender": "male"
}
{
    "product_name": "Men Cotton Solid Full Sleeve Casual Shirt (Available in Plus Size)",
    "product_id": "B07KXB3QC3",
    "price": "549",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo3NjYwNTk4ODg1MDg1MDUxOjE3MDg1MzI2NDg6c3BfYXRmX25leHQ6MzAwMDQzNjIzOTYzNDMyOjowOjo&url=%2FAmazon-Brand-Symbol-Regular-AZ-SY-RR-12A-04_Maroon_Small%2Fdp%2FB07KXB3QC3%2Fref%3Dsr_1_4658%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%26qid

INFO:root:Page 99 data collected
INFO:root:All data saved to men shirts .json
INFO:root:All data saved to mensirt1.csv


{
    "product_name": "Men's Shirt",
    "product_id": "B0C8MQDL9L",
    "price": "1,305",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1MDg5NDk3MDY1MTc1MDUxOjE3MDg1MzI2NTg6c3BfYXRmX25leHQ6MzAwMTAzMDQ4NDUzMjMyOjowOjo&url=%2FVan-Hesuen-Regular-Shirt-VHSHMCUPO42224_Beige%2Fdp%2FB0C8MQDL9L%2Fref%3Dsr_1_4705%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%26qid%3D1708532657%26sprefix%3Dshirts%252Caps%252C434%26sr%3D8-4705-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "Van Heusen",
    "gender": "male"
}
{
    "product_name": "Mens 100% Cotton Full Sleeve Regular Fit Fashion Checkered Shirt",
    "product_id": "B0BW3P1CJM",
    "price": "719",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1MDg5NDk3MDY1MTc1MDUxOjE3MDg1MzI2NTg6c3BfYXRmX25leHQ6MzAwMTMzMzcyODIyMDMyOjowOjo&url=%2FGLORYBOYZ-Cotton-Regular-Fashion-Checkered%2Fdp%2FB0BW3P1CJM%2Fref%3Dsr_1_4706%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%2

In [3]:
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json
import logging
import concurrent.futures
import pandas as pd

# Set up logging
logging.basicConfig(level=logging.INFO)

def get_random_user_agent():
    ua = UserAgent()
    return ua.random

def scrape_product(shoe):
    try:
        name_element = WebDriverWait(shoe, 20).until(EC.presence_of_element_located((By.XPATH, ".//span[@class='a-size-base-plus a-color-base a-text-normal']")))
        name = name_element.text
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product name")
        name = "No name available"

    try:
        product_id = shoe.get_attribute("data-asin")
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product ID")
        product_id = ""

    try:
        price_element = shoe.find_element(By.XPATH, ".//span[@class='a-price-whole']")
        price = price_element.text if price_element else "0"
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product price")
        price = "No price available"

    try:
        product_url_element = shoe.find_element(By.XPATH, ".//a[@class='a-link-normal s-no-outline']")
        product_url = product_url_element.get_attribute("href") if product_url_element else ""
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product URL")
        product_url = ""

    try:
        brand_element = shoe.find_element(By.XPATH, ".//span[@class='a-size-base-plus a-color-base']")
        brand = brand_element.text.strip()
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product description")
        brand = "No description available"

    return {
        "product_name": name,
        "product_id": product_id,
        "price": price,
        "product_url": product_url,
        "brand_name": brand,
        "gender": "male"  # Hardcoded gender as female
    }

def main():
    # Use the Firefox driver
    options = webdriver.FirefoxOptions()
    driver = webdriver.Firefox(options=options)

    try:
        all_data = []  # List to store data from all pages

        for page_number in range(101, 110): 
            url = f"https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page={page_number}"
            logging.info(f"Scraping page {page_number}: {url}")

            # Setting random user agent for each request
            user_agent = get_random_user_agent()
            logging.info(f"Using user agent: {user_agent}")
            options.set_preference("general.useragent.override", user_agent)

            driver.get(url)
            driver.maximize_window()

            # Add a delay to allow the page to load
            time.sleep(2)

            # Adjusted XPath expression to capture all product elements
            shoes = driver.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')

            data = []

            # Concurrently scrape product information
            with concurrent.futures.ThreadPoolExecutor() as executor:
                results = executor.map(scrape_product, shoes)

            # Collect results
            for result in results:
                data.append(result)
                print(json.dumps(result, indent=4))

            all_data.extend(data)

            logging.info(f"Page {page_number} data collected")

        # Save all data to a single JSON file
        with open("menshirts3.json", "w") as json_file:
            json.dump(all_data, json_file, indent=4)
        logging.info("All data saved to men shirts .json")

        # Convert all data to a DataFrame
        df = pd.DataFrame(all_data)
        df.to_csv("menshirts3.csv", index=False)
        logging.info("All data saved to mensirt1.csv")

    finally:
        driver.quit()

if __name__ == "__main__":
    main()


INFO:root:Scraping page 101: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=101
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/117.0
INFO:root:Page 101 data collected
INFO:root:Scraping page 102: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=102
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/115.0
INFO:root:Page 102 data collected
INFO:root:Scraping page 103: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=103
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36


{
    "product_name": "Men's Cotton Rich Solid Polo T-Shirt Regular Fit (Available in Plus Size)",
    "product_id": "B07MZKNLGF",
    "price": "445",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1MTI1ODUzNzA2MjIwMDUxOjE3MDg1MzI3Nzk6c3BfYXRmX25leHQ6MzAwMTE2ODI5NDE0NDMyOjowOjo&url=%2FAmazon-Brand-Symbol-Polo_SS19PLS2_Fog-Teal_Medium%2Fdp%2FB07MZKNLGF%2Fref%3Dsr_1_4849_sspa%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%26qid%3D1708532779%26sprefix%3Dshirts%252Caps%252C434%26sr%3D8-4849-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "Amazon Brand - Symbol",
    "gender": "male"
}
{
    "product_name": "Men Cotton Solid Full Sleeve Casual Shirt (Available in Plus Size)",
    "product_id": "B0822PSL3C",
    "price": "549",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1MTI1ODUzNzA2MjIwMDUxOjE3MDg1MzI3Nzk6c3BfYXRmX25leHQ6MzAwMDQzNjIzOTY0MzMyOjowOjo&url=%2FAmazon-Brand-Symbol-Regular-AZ-SY-RR-12A-10_Amaranth%

INFO:root:Page 103 data collected
INFO:root:Scraping page 104: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=104
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36


{
    "product_name": "Black Bull Men's Regular Fit Cotton Casual Shirt for Men Full Sleeves",
    "product_id": "B0CRHPZSVT",
    "price": "799",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MToxODgwODM2NjI0Nzk2MDUxOjE3MDg1MzI3OTA6c3BfYXRmX25leHQ6MzAwMTIxMTk0MzcyMTMyOjowOjo&url=%2FRegular-Cotton-Casual-Sleeves-Medium%2Fdp%2FB0CRHPZSVT%2Fref%3Dsr_1_4897_sspa%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%26qid%3D1708532790%26sprefix%3Dshirts%252Caps%252C434%26sr%3D8-4897-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "Generic",
    "gender": "male"
}
{
    "product_name": "Men's Collared Button Up Shirt for Casual Wear | Premium Stretchable Cotton Shirt Crafted with Regular Sleeve, Comfort Fit & High-Performance for Everyday Wear",
    "product_id": "B0BWRFZBP3",
    "price": "645",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MToxODgwODM2NjI0Nzk2MDUxOjE3MDg1MzI3OTA6c3BfYXRmX25leHQ6MjAxMjQ3MDkwMjM3OTg6OjA6Og

INFO:root:Page 104 data collected
INFO:root:Scraping page 105: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=105
INFO:root:Using user agent: Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/116.0


...

{
    "product_name": "Men Cotton Solid Full Sleeve Casual Shirt (Available in Plus Size)",
    "product_id": "B0822Q4TDC",
    "price": "549",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MToxMzYxMDUzNzc2MDg1MDUxOjE3MDg1MzI4MzM6c3BfYXRmX25leHQ6MzAwMDQzNjIzOTY1OTMyOjowOjo&url=%2FAmazon-Brand-Symbol-Regular-AZ-SY-RR-12A-10_Amaranth%2Fdp%2FB0822Q4TDC%2Fref%3Dsr_1_5089_sspa%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%26qid%3D1708532833%26sprefix%3Dshirts%252Caps%252C434%26sr%3D8-5089-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "Amazon Brand - Symbol",
    "gender": "male"
}
{
    "product_name": "Men's Cotton Solid Full Sleeve Mandarin Collar Casual Shirt (Available in Plus Size)",
    "product_id": "B08JR29BJS",
    "price": "599",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MToxMzYxMDUzNzc2MDg1MDUxOjE3MDg1MzI4MzM6c3BfYXRmX25leHQ6MzAwMDQzNjIzOTYxOTMyOjowOjo&url=%2FAmazon-Brand-Symbol-Regular-AW-SY-MCS-1

INFO:root:Page 108 data collected
INFO:root:Scraping page 109: https://www.amazon.in/s?k=shirts+for+men&crid=2GV9PJUGY6RH9&sprefix=shirts%2Caps%2C434&ref=nb_sb_ss_ts-doa-p_1_6&page=109
INFO:root:Using user agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/117.0


{
    "product_name": "Mens 100% Cotton Full Sleeve Regular Fit Fashion Checkered Shirt",
    "product_id": "B0BVFXLT96",
    "price": "719",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTozOTA2OTM4ODk3OTQ5MDUxOjE3MDg1MzI4NDI6c3BfYXRmX25leHQ6MzAwMTMzMzcyODIwODMyOjowOjo&url=%2FGLORYBOYZ-Cotton-Regular-Fashion-Checkered%2Fdp%2FB0BVFXLT96%2Fref%3Dsr_1_5137_sspa%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%26qid%3D1708532842%26sprefix%3Dshirts%252Caps%252C434%26sr%3D8-5137-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "GLORYBOYZ",
    "gender": "male"
}
{
    "product_name": "Men's Collared Button Up Shirt for Casual Wear | Premium Stretchable Cotton Shirt Crafted with Regular Sleeve, Comfort Fit & High-Performance for Everyday Wear",
    "product_id": "B0BWRD4Q7Q",
    "price": "569",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTozOTA2OTM4ODk3OTQ5MDUxOjE3MDg1MzI4NDI6c3BfYXRmX25leHQ6MjAxMjQ3MDkwMjMzOTg6OjA

INFO:root:Page 109 data collected
INFO:root:All data saved to men shirts .json
INFO:root:All data saved to mensirt1.csv


{
    "product_name": "Mens 100% Cotton Full Sleeve Regular Fit Fashion Checkered Shirt",
    "product_id": "B0BW3SFC9P",
    "price": "719",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1MzkzMzQyNDg3NzQyMDUxOjE3MDg1MzI4NTA6c3BfYXRmX25leHQ6MzAwMTMzMzcyODIyMTMyOjowOjo&url=%2FGLORYBOYZ-Cotton-Regular-Fashion-Checkered%2Fdp%2FB0BW3SFC9P%2Fref%3Dsr_1_5185_sspa%3Fcrid%3D2GV9PJUGY6RH9%26keywords%3Dshirts%2Bfor%2Bmen%26qid%3D1708532850%26sprefix%3Dshirts%252Caps%252C434%26sr%3D8-5185-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "GLORYBOYZ",
    "gender": "male"
}
{
    "product_name": "Men's Cotton Chequered Slim Fit Formal Shirt",
    "product_id": "B0B5YBSNPM",
    "price": "1,139",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1MzkzMzQyNDg3NzQyMDUxOjE3MDg1MzI4NTA6c3BfYXRmX25leHQ6MzAwMTAzMDY2NTQ0ODMyOjowOjo&url=%2FVan-Heusen-Mens-Shirt-VHSFRSLBX71474_Multi%2Fdp%2FB0B5YBSNPM%2Fref%3Dsr_1_5186_sspa%3Fcrid%3D2GV9PJUG